In [1]:
import os
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "Enter your key"
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11180\3304190155.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")


In [2]:
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchResults 
import json


@tool
def search_web_tool(query: str):
    """
    Searches the web and returns results.
    """
    search_tool = DuckDuckGoSearchResults(num_results=10, verbose=True)
    return search_tool.run(query)


In [3]:
from crewai import Agent

# Agent Resercher
guide_expert= Agent( 
    role="City Local Guide Expert",
    goal="Provides information on things to do in the city based on the user's interests.",
    backstory="""A local expert with a passion for sharing the best experiences and hidden gems of their city.""",
    tools=[search_web_tool],
    verbose=True,
    max_iter=5,
    llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    allow_delegation=False,
    )

In [4]:
# Agent city expert
location_expert = Agent(
    role="Travel Trip Expert",
    goal="Adapt to the user destination vity language (French if city in French Country. Gather helpful information about to the city and city during travel.",
    backstory="""A seasoned traveler who has explored various destinations and knows the ins and outs of travel logistics.""",
    tools=[search_web_tool],
    verbose=True,
    max_iter=5,
    llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    allow_delegation=False,
    )


In [5]:
planner_expert = Agent(
    role="Travel Planning Expert",
    goal="Compiles all gathered information to provide a comprehensive travel plan.",
    backstory="""
    You are a professional guide with a passion for travel.
    An organizational wizard who can turn a list of possibilities into a seamless itinerary.
    """,
    tools=[search_web_tool],
    verbose=True,
    max_iter=5,
    llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    allow_delegation=False,
    )


In [6]:

from crewai import Task

from_city = "India"
destination_city = "Rome"
date_from = "1st March 2025"
date_to = "7th March 2025"
interests = "sight seeing and good food"

location_task = Task(
    description=f"""
    In French : This task involves a comprehensive data collection process to provide the traveler with essential information about their destination. It includes researching and compiling details on various accommodations, ranging from budget-friendly hostels to luxury hotels, as well as estimating the cost of living in the area. The task also covers transportation options, visa requirements, and any travel advisories that may be relevant.
    consider also the weather conditions forcast on the travel dates. and all the events that may be relevant to the traveler during the trip period.
    
    Traveling from : {from_city}
    Destination city : {destination_city}
    Arrival Date : {date_from}
    Departure Date : {date_to}

    Follow this rules : 
    1. if the {destination_city} is in a French country : Respond in FRENCH.
    """,
    expected_output=f"""
    if the {destination_city} is in a French country : Respond in FRENCH.
    In markdown format : A detailed markdown report that includes a curated list of recommended places to stay, a breakdown of daily living expenses, and practical travel tips to ensure a smooth journey.
    """,
    agent=location_expert,
    output_file='city_report.md',
)





In [7]:
guide_task = Task(
    description=f"""
    if the {destination_city} is in a French country : Respond in FRENCH.
    Tailored to the traveler's personal {interests}, this task focuses on creating an engaging and informative guide to the city's attractions. It involves identifying cultural landmarks, historical spots, entertainment venues, dining experiences, and outdoor activities that align with the user's preferences such {interests}. The guide also highlights seasonal events and festivals that might be of interest during the traveler's visit.
    Destination city : {destination_city}
    interests : {interests}
    Arrival Date : {date_from}
    Departure Date : {date_to}

    Follow this rules : 
    1. if the {destination_city} is in a French country : Respond in FRENCH.
    """,
    expected_output=f"""
    An interactive markdown report that presents a personalized itinerary of activities and attractions, complete with descriptions, locations, and any necessary reservations or tickets.
    """,

    agent=guide_expert,
    output_file='guide_report.md',
)


In [8]:
planner_task = Task(
    description=f"""
    This task synthesizes all collected information into a detaileds introduction to the city (description of city and presentation, in 3 pragraphes) cohesive and practical travel plan. and takes into account the traveler's schedule, preferences, and budget to draft a day-by-day itinerary. The planner also provides insights into the city's layout and transportation system to facilitate easy navigation.
    Destination city : {destination_city}
    interests : {interests}
    Arrival Date : {date_from}
    Departure Date : {date_to}

    Follow this rules : 
    1. if the {destination_city} is in a French country : Respond in FRENCH.
    """,
    expected_output="""
    if the {destination_city} is in a French country : Respond in FRENCH.
    A rich markdown document with emojis on each title and subtitle, that :
    In markdown format : 
    # Welcome to {destination_city} :
    A 4 paragraphes markdown formated including :
    - a curated articles of presentation of the city, 
    - a breakdown of daily living expenses, and spots to visit.
    # Here's your Travel Plan to {destination_city} :
    Outlines a daily detailed travel plan list with time allocations and details for each activity, along with an overview of the city's highlights based on the guide's recommendations
    """,
    context=[location_task, guide_task],
    #context=context,
    agent=planner_expert,
        output_file='travel_plan.md',
        )


In [9]:
# Task: Location
def location_task(agent, from_city, destination_city, date_from, date_to):
    return Task(
        description=f"""
        In French : This task involves a comprehensive data collection process to provide the traveler with essential information about their destination. It includes researching and compiling details on various accommodations, ranging from budget-friendly hostels to luxury hotels, as well as estimating the cost of living in the area. The task also covers transportation options, visa requirements, and any travel advisories that may be relevant.
        consider also the weather conditions forcast on the travel dates. and all the events that may be relevant to the traveler during the trip period.
        
        Traveling from : {from_city}
        Destination city : {destination_city}
        Arrival Date : {date_from}
        Departure Date : {date_to}

        Follow this rules : 
        1. if the {destination_city} is in a French country : Respond in FRENCH.
        """,
        expected_output=f"""
        if the {destination_city} is in a French country : Respond in FRENCH.
        In markdown format : A detailed markdown report that includes a curated list of recommended places to stay, a breakdown of daily living expenses, and practical travel tips to ensure a smooth journey.
        """,
        agent=agent,
        output_file='city_report.md',
    )

# Task: Location
def guide_task(agent, destination_city, interests, date_from, date_to):    
    return Task(
        description=f"""
        if the {destination_city} is in a French country : Respond in FRENCH.
        Tailored to the traveler's personal {interests}, this task focuses on creating an engaging and informative guide to the city's attractions. It involves identifying cultural landmarks, historical spots, entertainment venues, dining experiences, and outdoor activities that align with the user's preferences such {interests}. The guide also highlights seasonal events and festivals that might be of interest during the traveler's visit.
        Destination city : {destination_city}
        interests : {interests}
        Arrival Date : {date_from}
        Departure Date : {date_to}

        Follow this rules : 
        1. if the {destination_city} is in a French country : Respond in FRENCH.
        """,
        expected_output=f"""
        An interactive markdown report that presents a personalized itinerary of activities and attractions, complete with descriptions, locations, and any necessary reservations or tickets.
        """,

        agent=agent,
        output_file='guide_report.md',
    )


# Task: Planner
def planner_task(context, agent, destination_city, interests, date_from, date_to):
    return Task(
        description=f"""
        This task synthesizes all collected information into a detaileds introduction to the city (description of city and presentation, in 3 pragraphes) cohesive and practical travel plan. and takes into account the traveler's schedule, preferences, and budget to draft a day-by-day itinerary. The planner also provides insights into the city's layout and transportation system to facilitate easy navigation.
        Destination city : {destination_city}
        interests : {interests}
        Arrival Date : {date_from}
        Departure Date : {date_to}

        Follow this rules : 
        1. if the {destination_city} is in a French country : Respond in FRENCH.
        """,
        expected_output="""
        if the {destination_city} is in a French country : Respond in FRENCH.
        A rich markdown document with emojis on each title and subtitle, that :
        In markdown format : 
        # Welcome to {destination_city} :
        A 4 paragraphes markdown formated including :
        - a curated articles of presentation of the city, 
        - a breakdown of daily living expenses, and spots to visit.
        # Here's your Travel Plan to {destination_city} :
        Outlines a daily detailed travel plan list with time allocations and details for each activity, along with an overview of the city's highlights based on the guide's recommendations
        """,
        #context=[location_task, guide_task],
        context=context,
        agent=agent,
        output_file='travel_plan.md',
        )


In [10]:

location_task = location_task(
  location_expert,
  from_city,
  destination_city,
  date_from,
  date_to
)

guide_task = guide_task(
  guide_expert,
  destination_city,
  interests,
  date_from,
  date_to
)

planner_task = planner_task(
  [location_task, guide_task],
  planner_expert,
  destination_city,
  interests,
  date_from,
  date_to,
)



In [12]:
from crewai import Crew, Process
crew = Crew(
agents=[location_expert, guide_expert, planner_expert],
tasks=[location_task, guide_task, planner_task],
process=Process.sequential,
full_output=True,
share_crew=False,
verbose=False
)

result = crew.kickoff()

# Agent: Travel Trip Expert
## Task: 
        In French : This task involves a comprehensive data collection process to provide the traveler with essential information about their destination. It includes researching and compiling details on various accommodations, ranging from budget-friendly hostels to luxury hotels, as well as estimating the cost of living in the area. The task also covers transportation options, visa requirements, and any travel advisories that may be relevant.
        consider also the weather conditions forcast on the travel dates. and all the events that may be relevant to the traveler during the trip period.
        
        Traveling from : India
        Destination city : Rome
        Arrival Date : 1st March 2025
        Departure Date : 7th March 2025

        Follow this rules : 
        1. if the Rome is in a French country : Respond in FRENCH.
        


C:\Users\ASUS\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


[snippet: Pour trouver le meilleur endroit où vous pourrez poser vos valises à Rome, les deux critères qui sont les plus importants sont votre budget et votre principal centre d'intérêt. Certains quartiers et établissements sont parfois très onéreux., title: Où dormir à Rome ? Les meilleurs quartiers et adresses où loger, link: https://www.voyageavecnous.fr/ou-dormir-a-rome/], [snippet: Il faut savoir qu'il n'y a pas forcément un seul et unique meilleur endroits où dormir à Rome. Le quartier le plus adapté dépendra de vous. De votre budget mais aussi de vos envies de découverte ou encore de la durée de votre séjour. Voici selon moi les critères majeurs à prendre en compte avant de savoir où loger à Rome :, title: Où dormir à Rome : quartiers & hébergements où loger - Voyage Way, link: https://www.voyageway.com/ou-dormir-a-rome], [snippet: Pour vous aider à trouver l'endroit idéal où dormir à Rome, je vous expose dans cet article les meilleurs quartiers où se loger à Rome. Vous y trouve

C:\Users\ASUS\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
C:\Users\ASUS\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


[snippet: En moyenne, le coût de la vie à Rome en 2025 est sensiblement identique à celui de la France. Le salaire mensuel moyen est de : 1727€ 1L d'essence coûte : 1.8€ Un déjeuner au restaurant : 15€ Le prix d'une place de cinéma est 10€ La location d'un appartement dans le centre-ville coûte 2120€/mois, title: Coût de la vie et Prix à Rome en 2025, pour vivre ou voyager, link: https://www.combien-coute.net/cout-de-la-vie/italie/rome/], [snippet: Calculez et optimisez votre budget voyage à Rome en comparant les prix : haute saison, coût de la vie sur place, tarifs des agences locales… tout y est pour anticiper votre budget. ... Mis à jour le 16-06-2025. Sommaire. 1. Prix d'un voyage sans agence; 2. Comparatif des prix avec une agence; 3. Prix des vols, title: Budget et prix d'un voyage à Rome - Partir.com, link: https://www.partir.com/Italie/rome-et-latium/cout-de-la-vie.html], [snippet: Coût de la vie en Rome est: 1.33 fois plus bas que dans Paris 1.19 fois plus bas que dans Nice ..

...

 

I encountered an error while trying to use the tool. This was the error: https://html.duckduckgo.com/html RuntimeError: error sending request for url (https://html.duckduckgo.com/html): operation timed out

Caused by:
    operation timed out.
 Tool search_web_tool accepts these inputs: Tool Name: search_web_tool
Tool Arguments: {'query': {'description': None, 'type': 'str'}}
Tool Description: 
    Searches the web and returns results.
    



# Agent: City Local Guide Expert
## Thought: Thought: I should search for the best sightseeing spots in Rome for the traveler's visit in March 2025 based on their interest in sightseeing.
## Using tool: search_web_tool
## Tool Input: 
"{\"query\": \"must-visit attractions in Rome for sightseeing\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: https://html.duckduckgo.com/html RuntimeError: error sending request for url (https://html.duckduckgo.com/html): operation timed out

Caused by:
    operation

C:\Users\ASUS\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
C:\Users\ASUS\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
C:\Users\ASUS\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
C:\Users\ASUS\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
C:\Users\ASUS\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
C:\Users\ASUS\anaconda3\Lib\site-packages\langchain_community\utilities\duckduck

 

I encountered an error while trying to use the tool. This was the error: https://lite.duckduckgo.com/lite/ RuntimeError: error sending request for url (https://lite.duckduckgo.com/lite/): operation timed out

Caused by:
    operation timed out.
 Tool search_web_tool accepts these inputs: Tool Name: search_web_tool
Tool Arguments: {'query': {'description': None, 'type': 'str'}}
Tool Description: 
    Searches the web and returns results.
    



# Agent: Travel Planning Expert
## Thought: Thought: I should search for information on events happening in Rome in March 2025 to enhance the travel plan.
## Using tool: search_web_tool
## Tool Input: 
"{\"query\": \"Events in Rome in March 2025\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: https://lite.duckduckgo.com/lite/ RuntimeError: error sending request for url (https://lite.duckduckgo.com/lite/): operation timed out

Caused by:
    operation timed out.
 Tool search_web_tool accepts these

C:\Users\ASUS\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
C:\Users\ASUS\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
C:\Users\ASUS\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
C:\Users\ASUS\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
C:\Users\ASUS\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
C:\Users\ASUS\anaconda3\Lib\site-packages\langchain_community\utilities\duckduck

 

I encountered an error while trying to use the tool. This was the error: https://lite.duckduckgo.com/lite/ RuntimeError: error sending request for url (https://lite.duckduckgo.com/lite/): operation timed out

Caused by:
    operation timed out.
 Tool search_web_tool accepts these inputs: Tool Name: search_web_tool
Tool Arguments: {'query': {'description': None, 'type': 'str'}}
Tool Description: 
    Searches the web and returns results.
    



# Agent: Travel Planning Expert
## Thought: Thought: I should search for information on events happening in Rome in March 2025 to enhance the travel plan.
## Using tool: search_web_tool
## Tool Input: 
"{\"query\": \"Events in Rome in March 2025\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: https://lite.duckduckgo.com/lite/ RuntimeError: error sending request for url (https://lite.duckduckgo.com/lite/): operation timed out

Caused by:
    operation timed out.
 Tool search_web_tool accepts these